Importation des librairies

In [20]:
from bs4 import BeautifulSoup
import requests as req
import re   
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import pandas as pd


**Création d'une fonction pour avoir le code source de chaque url, les valeur de (dict_url sont une idée de note avortée)**

In [21]:
dict_of_urls ={'https://fr.quora.com/Comment-comprendre-les-maths': 60,
 'https://fr.quora.com/Quel-est-le-Pokémon-le-plus-fort': 60,
 'https://fr.quora.com/Que-pensez-vous-des-jeux-vidéos': 60,
 'https://fr.quora.com/Quest-ce-que-les-jeux-vidéo-vous-ont-apporté-dans-la-vie': 75,
 'https://fr.quora.com/Comment-être-le-meilleur-dans-les-jeux-vidéo': 70,
 'https://fr.quora.com/Selon-vous-le-jeu-vidéo-est-il-de-lart': 55,
 'https://fr.quora.com/Pour-vous-qu’est-ce-que-l’art': 60,
 'https://fr.quora.com/Pourquoi-aimez-vous-lart': 75}

def get_soup(url):
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(15)
    cookie_button = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    cookie_button.click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    return soup


**Création d'une fonction pour avoir les questions et les réponses**

In [22]:
def get_question(soup):
    
    question = soup.find_all('div', class_='q-box qu-mb--tiny')
    questions = [question.get_text() for question in question]
    for i in range(len(questions)):
        if  questions[i].startswith('Associé'):
                questions[i] = questions[i][7:]
    return questions     

In [23]:
#fonction pour avoir les réponses. Arriver sur un post, on clique sur "Continuer la lecture" pour avoir la réponse complète. On scroll jusqu'à la fin de la page pour avoir toutes les réponses.
def get_all_answers(url):
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(5)
    cookie_button = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    cookie_button.click()
    time.sleep(5)
    #click on all button with "continuer la lecture"
    buttons = driver.find_elements(By.XPATH, '//*[@class="q-box qu-mb--tiny"]')
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    answers = soup.find_all('p', class_="q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start", style="box-sizing: border-box; margin-bottom: 1em; overflow-wrap: anywhere; direction: ltr;")
    answers = [answer.get_text() for answer in answers]
    answers = pd.Series(answers).unique().tolist()
    answers = list(set(answers))
    for answer in answers:
        if answer.startswith('Q'):
            answers.remove(answer)
        elif len(answer) < 20:
            answers.remove(answer)
        else :
            pass
    return answers


**Création du dataframe Quora**

In [24]:
import pandas as pd
dataframes = []

for url, nb in dict_of_urls.items():
    soup = get_soup(url)
    questions = get_question(soup)
    answers = get_all_answers(url)
    for answer in answers:
        for question in questions:
            df = pd.DataFrame({'Question': [question], 'Réponse': [answer], 'URL': [url]})
            dataframes.append(df)

# Concatenate all the dataframes
df = pd.concat(dataframes, ignore_index=True)
df

    




,Question,Réponse,URL
0,Comment comprendre intuitivement les mathémati...,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths
1,Pourquoi je n’arrive pas à comprendre les maths ?,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths
2,Comment comprendre intuitivement les mathémati...,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths
3,Pourquoi je n’arrive pas à comprendre les maths ?,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths
4,Comment comprendre intuitivement les mathémati...,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths
...,...,...,...
6292,"Pour vous, qu’est-ce que l’art ?","Si vous n’avez jamais vu La linea (Cava, Osval...",https://fr.quora.com/Pourquoi-aimez-vous-lart
6293,Quelle est l'utilité de l'art ?,"Si vous n’avez jamais vu La linea (Cava, Osval...",https://fr.quora.com/Pourquoi-aimez-vous-lart
6294,Pourquoi l'appreciation de l'art est-elle impo...,"Si vous n’avez jamais vu La linea (Cava, Osval...",https://fr.quora.com/Pourquoi-aimez-vous-lart
6295,Pourquoi est-ce que les humains aimons l'art ?,"Si vous n’avez jamais vu La linea (Cava, Osval...",https://fr.quora.com/Pourquoi-aimez-vous-lart


In [25]:
#afficher une réponse du dataframe
len(df['Réponse'].unique())

1083

**Fonction pour créer le score de fiabilité final sur des critères permettant réelement au modèle de s'améliorer**

In [50]:
import language_tool_python
from spellchecker import SpellChecker
import nltk
import spacy



def calculate_reliability_score(answer):
    # Spelling, Grammar, and Syntax Errors
    doc = nlp(answer)
    spelling_errors = len(spell.unknown(answer.split()))
    grammar_errors = len(tool.check(answer))
    syntax_errors = 0
    for token in doc:
        if token.dep_ == 'ROOT' and token.pos_ == 'VERB':
            syntax_errors += 1

    # Word Count
    word_count = len(nltk.word_tokenize(answer))
    max_word_count = 100

    # Presence of Sources
    scores_sources = {}

    # Parcourir les entités nommées dans le texte
    for entite in doc.ents:
        # Vérifier si l'entité est de type ORG (organisation)
        if entite.label_ == "ORG":
            nom_source = entite.text
            # Incrémenter le score de la source dans le dictionnaire
            if nom_source not in scores_sources:
                scores_sources[nom_source] = 0.0
            scores_sources[nom_source] += 1.0 / len(doc.ents)

    # Calculate reliability score
    spelling_weight = 0.4
    grammar_weight = 0.3
    syntax_weight = 0.2
    word_count_weight = 0.1
    sources_weight = 0.1

    reliability_score = (
        spelling_weight * (1 - spelling_errors / word_count) +
        grammar_weight * (1 - grammar_errors / word_count) +
        syntax_weight * (1 - syntax_errors / word_count) +
        word_count_weight * (word_count / max_word_count) +
        sources_weight * len(scores_sources)    
    )

    return reliability_score

# Load the language model
nlp = spacy.load('fr_core_news_md')
# Load the spell checker
spell = SpellChecker(language='fr')
# Load the grammar checker
tool = language_tool_python.LanguageTool('fr')






**Rajout des thèmes dans le dataset**

In [52]:
jeu_keywords = ['jeu', 'gaming', 'console', 'videogame','pokémon']  # Ajoutez vos mots-clés ici

df['Thème'] = df['Question'].apply(lambda x: 'Mathématiques' if 'math' in x.lower() 
                                   else 'Jeux vidéo' if any(keyword in x.lower() for keyword in jeu_keywords) 
                                   else 'Art' if 'art' in x.lower() 
                                   else 'Autre')
df.head(20)

,Question,Réponse,URL,Thème
0,Comment comprendre intuitivement les mathémati...,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
1,Pourquoi je n’arrive pas à comprendre les maths ?,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
2,Comment comprendre intuitivement les mathémati...,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
3,Pourquoi je n’arrive pas à comprendre les maths ?,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
4,Comment comprendre intuitivement les mathémati...,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
5,Pourquoi je n’arrive pas à comprendre les maths ?,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
6,Comment comprendre intuitivement les mathémati...,"ce qui est ironique, c'est que j'ai un fils po...",https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
7,Pourquoi je n’arrive pas à comprendre les maths ?,"ce qui est ironique, c'est que j'ai un fils po...",https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
8,Comment comprendre intuitivement les mathémati...,le fait qu'on m'aie seriné dès le primaire que...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
9,Pourquoi je n’arrive pas à comprendre les maths ?,le fait qu'on m'aie seriné dès le primaire que...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques


***Rajout d'autres dataset scrapé par les autres membres du groupes***

In [53]:
df_urbanisme = pd.read_csv('faq_urbanisme.csv')
df_urbanisme['Thème'] = 'Urbanisme'
df_urbanisme.head(20)

,Questions,Réponses,Thème
0,Quelle est l’évolution d’un quartier et ses fu...,"Depuis avril 2019, toutes vos démarches peuven...",Urbanisme
1,Les travaux en cours sont-ils autorisés? Un vo...,"Si vous ne disposez pas d’un accès numérique, ...",Urbanisme
2,Nuisances sonores sur la voie publique?,S'il n'existe pas de panneau affichant une aut...,Urbanisme
3,Vous souhaitez signaler des travaux de transfo...,"Concernant les projets de la ville de Paris, l...",Urbanisme
4,Quels sont les droits des locataires ou des co...,"En tout état de cause, les futurs projets, pri...",Urbanisme
5,Qu'est-ce que l'usage et la destination d’un l...,S’il existe unpanneau affichant le numéro du P...,Urbanisme
6,Quand et comment faire un changement d’usage (...,S'il n'existe pasun panneau affichant le numér...,Urbanisme
7,Comment obtenir un descriptif de la servitude ...,Vous pouvez lessignaler soit:,Urbanisme
8,Quand et comment faut-il faire une déclaration...,Il est nécessaire de mentionner lestravaux con...,Urbanisme
9,Droit de préemption urbain simple ou renforcé ...,Retrouver tous les contacts en cas de problème...,Urbanisme


In [54]:
df_urbanisme = df_urbanisme.rename(columns={'Réponses': 'Réponse'})
df_urbanisme = df_urbanisme.rename(columns={'Questions': 'Question'})
df_urbanisme['URL'] = ['https://faq.urbanisme.gouv.fr/faq/urbanisme-et-foncier/urbanisme-et-foncier-generalites/urbanisme-et-foncier-generalites-1' for i in range(len(df_urbanisme))]
df_urbanisme.head(20)

,Question,Réponse,Thème,URL
0,Quelle est l’évolution d’un quartier et ses fu...,"Depuis avril 2019, toutes vos démarches peuven...",Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
1,Les travaux en cours sont-ils autorisés? Un vo...,"Si vous ne disposez pas d’un accès numérique, ...",Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
2,Nuisances sonores sur la voie publique?,S'il n'existe pas de panneau affichant une aut...,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
3,Vous souhaitez signaler des travaux de transfo...,"Concernant les projets de la ville de Paris, l...",Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
4,Quels sont les droits des locataires ou des co...,"En tout état de cause, les futurs projets, pri...",Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
5,Qu'est-ce que l'usage et la destination d’un l...,S’il existe unpanneau affichant le numéro du P...,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
6,Quand et comment faire un changement d’usage (...,S'il n'existe pasun panneau affichant le numér...,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
7,Comment obtenir un descriptif de la servitude ...,Vous pouvez lessignaler soit:,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
8,Quand et comment faut-il faire une déclaration...,Il est nécessaire de mentionner lestravaux con...,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...
9,Droit de préemption urbain simple ou renforcé ...,Retrouver tous les contacts en cas de problème...,Urbanisme,https://faq.urbanisme.gouv.fr/faq/urbanisme-et...


In [55]:
df = pd.concat([df, df_urbanisme], ignore_index=True)

In [56]:
df_cg = pd.read_csv('cg.csv')
df_cg['Thème'] = 'Culture générale'
df_cg['URL'] = [None for i in range(len(df_cg))]
df_cg.head(20)

,Question,Réponse,Thème,URL
0,Comment appelle-t-on une bande de corbeaux ?,Un meurtre,Culture générale,None
1,Quel est le nom d’un duel à trois ?,Un truel,Culture générale,None
2,Quel est l’animal le plus fort par rapport à s...,Bousiers,Culture générale,None
3,Qu’est-il interdit de manger avec une fourchet...,Poulet frit,Culture générale,None
4,Combien de points compte un jeu de dés ?,42,Culture générale,None
5,L’Islande a modifié les itinéraires de circula...,Elfes,Culture générale,None
6,Quelle partie du corps est la seule à être com...,Yeux,Culture générale,None
7,Qui a inventé le mot « vomi » ?,William Shakespeare,Culture générale,None
8,Qu’est-ce que la peur de l’acrophobie ?,Voler,Culture générale,None
9,Qu’est-ce qui peut être brisé mais ne peut jam...,Une promesse,Culture générale,None


In [57]:
df = pd.concat([df, df_cg], ignore_index=True)
df

,Question,Réponse,URL,Thème
0,Comment comprendre intuitivement les mathémati...,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
1,Pourquoi je n’arrive pas à comprendre les maths ?,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
2,Comment comprendre intuitivement les mathémati...,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
3,Pourquoi je n’arrive pas à comprendre les maths ?,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
4,Comment comprendre intuitivement les mathémati...,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
...,...,...,...,...
6368,Combien y a-t-il d’os dans le corps humain ?,206,None,Culture générale
6369,Combien de pays font partie de l’Union europée...,27 pays européens,None,Culture générale
6370,"Le Machu Picchu, l’une des sept nouvelles merv...",Pérou,None,Culture générale
6371,Quelle chaîne de restauration rapide est célèb...,McDonald’s,None,Culture générale


In [58]:
df_out = pd.read_csv('out.csv')
df_out['Thème'] = 'Vie publique'
df_out['URL']= ['https://www.vie-publique.fr/questions-reponses' for i in range(len(df_out))]
df_out.head(20)

,Question,Réponse,Thème,URL
0,Programmation pluriannuelle de l’énergie (PPE)...,La loi de transition énergétique pour la crois...,Vie publique,https://www.vie-publique.fr/questions-reponses
1,Pourquoi viser la neutralité carbone à l’horiz...,L’objectif de la neutralité carbone est de lut...,Vie publique,https://www.vie-publique.fr/questions-reponses
2,Quels sont les objectifs de la stratégie sur l...,Dans le cadre de la consultation publique qui ...,Vie publique,https://www.vie-publique.fr/questions-reponses
3,Quelles mesures contient la PPE pour accompagn...,La nouvelle PPE vise à réduire la consommation...,Vie publique,https://www.vie-publique.fr/questions-reponses
4,Quels sont les objectifs de la dernière SNBC a...,"Adoptée par décret en 2015, la Stratégie natio...",Vie publique,https://www.vie-publique.fr/questions-reponses
5,Qu’appelle-t-on bureau de vote ?,Le bureau de vote désigne à la fois le local o...,Vie publique,https://www.vie-publique.fr/questions-reponses
6,Comment est constitué le bureau de vote ?,"Chaque bureau est composé :\nd’un président, q...",Vie publique,https://www.vie-publique.fr/questions-reponses
7,Le vote peut-il avoir lieu en l'absence d'asse...,"Si le jour du vote, le nombre d'assesseurs est...",Vie publique,https://www.vie-publique.fr/questions-reponses
8,Combien de bureaux de vote par commune ?,Les bureaux de vote sont institués par un arrê...,Vie publique,https://www.vie-publique.fr/questions-reponses
9,L’électeur peut-il choisir son bureau de vote ?,Les électeurs n’ont pas le choix de leur burea...,Vie publique,https://www.vie-publique.fr/questions-reponses


In [59]:
df = pd.concat([df, df_out], ignore_index=True)
df

,Question,Réponse,URL,Thème
0,Comment comprendre intuitivement les mathémati...,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
1,Pourquoi je n’arrive pas à comprendre les maths ?,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
2,Comment comprendre intuitivement les mathémati...,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
3,Pourquoi je n’arrive pas à comprendre les maths ?,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
4,Comment comprendre intuitivement les mathémati...,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques
...,...,...,...,...
6411,Qu'en est-il de l'érosion côtière ?,"Le littoral est exposé, à l’érosion côtière. S...",https://www.vie-publique.fr/questions-reponses,Vie publique
6412,Comment la biodiversité a-t-elle évolué ?,"Globalement, en 2023, 16% des espèces évaluées...",https://www.vie-publique.fr/questions-reponses,Vie publique
6413,La France est-elle particulièrement touchée pa...,"Entre 1982 et 2022, chaque année, près de 6 00...",https://www.vie-publique.fr/questions-reponses,Vie publique
6414,Quelle est l'efficacité énergétique de la Fran...,"L’intensité énergétique, qui mesure le rapport...",https://www.vie-publique.fr/questions-reponses,Vie publique


**Calcul du score de fiabilité puis enregistrement du dataset final au format CSV**

In [60]:
df['Score de fiabilité'] = pd.Series(df['Réponse'].unique()).apply(calculate_reliability_score)
df['Score de fiabilité'] = df['Score de fiabilité'].fillna(df.groupby('Réponse')['Score de fiabilité'].transform('first'))
df['Score de fiabilité'] = df['Score de fiabilité'].fillna(df['Score de fiabilité'].mean())
df

,Question,Réponse,URL,Thème,Score de fiabilité
0,Comment comprendre intuitivement les mathémati...,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques,0.877857
1,Pourquoi je n’arrive pas à comprendre les maths ?,Ce qu’il faut faire c’est identifié les règles...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques,0.872000
2,Comment comprendre intuitivement les mathémati...,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques,0.862444
3,Pourquoi je n’arrive pas à comprendre les maths ?,À chacun de trouver sa voie. Parmi presque tou...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques,0.898083
4,Comment comprendre intuitivement les mathémati...,On entend souvent de brillants mathématiciens ...,https://fr.quora.com/Comment-comprendre-les-maths,Mathématiques,0.804250
...,...,...,...,...,...
6411,Qu'en est-il de l'érosion côtière ?,"Le littoral est exposé, à l’érosion côtière. S...",https://www.vie-publique.fr/questions-reponses,Vie publique,0.870499
6412,Comment la biodiversité a-t-elle évolué ?,"Globalement, en 2023, 16% des espèces évaluées...",https://www.vie-publique.fr/questions-reponses,Vie publique,0.870499
6413,La France est-elle particulièrement touchée pa...,"Entre 1982 et 2022, chaque année, près de 6 00...",https://www.vie-publique.fr/questions-reponses,Vie publique,0.870499
6414,Quelle est l'efficacité énergétique de la Fran...,"L’intensité énergétique, qui mesure le rapport...",https://www.vie-publique.fr/questions-reponses,Vie publique,0.870499


In [70]:
df_entreprise = pd.read_csv('faq_entreprises_gouv.csv')
df_entreprise['Thème'] = 'Entreprise'
df_entreprise['Score de fiabilité'] = df_entreprise['Réponse'].apply(calculate_reliability_score)
df_entreprise

,Question,Réponse,URL,Thème,Score de fiabilité
0,Où trouver des études sur le marché de l'embal...,Vous pouvez trouver des informations relatives...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,0.977884
1,Où trouver des statistiques sur la création de...,La DGE n’est pas en mesure de répondre sur un ...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,1.060471
2,Est-il possible de diffuser les études économi...,Vous pouvez tout à fait relayer les informatio...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,0.965565
3,Où trouver les chiffres concernant la création...,Nous vous invitons à vous adresser à l'Institu...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,1.067286
4,"Je suis micro-entrepreneur, suis-je assujetti ...","Les micro-entrepreneurs, qu'ils possèdent un l...",https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,1.322242
5,Où envoyer la déclaration de la retenue à la s...,Cette question ne relève pas de la compétence ...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,1.300000
6,NaN,Les micro-entrepreneurs exerçant une activité ...,https://www.entreprises.gouv.fr/fr/faq?page=0,Entreprise,1.170625
7,Où trouver des études sur le marché de l'embal...,Vous pouvez trouver des informations relatives...,https://www.entreprises.gouv.fr/fr/faq?page=1,Entreprise,0.977884
8,Où trouver des statistiques sur la création de...,La DGE n’est pas en mesure de répondre sur un ...,https://www.entreprises.gouv.fr/fr/faq?page=1,Entreprise,1.060471
9,Est-il possible de diffuser les études économi...,Vous pouvez tout à fait relayer les informatio...,https://www.entreprises.gouv.fr/fr/faq?page=1,Entreprise,0.965565


In [71]:
df_JO = pd.read_csv('JO.csv')
df_JO['Thème'] = 'Jeux Olympiques'
df_JO['Score de fiabilité'] = df_JO['Réponse'].apply(calculate_reliability_score)
df_JO


,Question,Réponse,URL,Thème,Score de fiabilité
0,Puis-je accéder à mon parking durant les Jeux ?,Si votre parking se situe dans un périmètre ro...,https://anticiperlesjeux.gouv.fr/faq,Jeux Olympiques,1.016029
1,Quelles sont les livraisons autorisées dans le...,Les livraisons autorisées dans les périmètres ...,https://anticiperlesjeux.gouv.fr/faq,Jeux Olympiques,1.209177
2,Quand seront activées les voies olympiques ?,Les voies olympiques seront activées sur les d...,https://anticiperlesjeux.gouv.fr/faq,Jeux Olympiques,0.991919
3,"Je suis PMR, comment puis-je me déplacer avec ...",Les personnes à mobilité réduite peuvent circu...,https://anticiperlesjeux.gouv.fr/faq,Jeux Olympiques,1.169188
4,Quels ponts resteront traversants le jour de l...,Le jour de la cérémonie d'ouverture des Jeux O...,https://anticiperlesjeux.gouv.fr/faq,Jeux Olympiques,0.867000
...,...,...,...,...,...
61,Quels sont les sites de compétitions temporair...,Certains sites de compétitions seront temporai...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.979585
62,Quels sont les sites de compétitions ?,"Il existe plus de 30 sites de compétition, rép...",https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,1.288634
63,Comment rester alerté des éventuels changement...,Inscrivez-vous à l’alerte info pour rester inf...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.969000
64,Où et quand auront lieu les épreuves des Jeux ?,Les Jeux Olympiques et Paralympiques de Paris ...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.893000


In [79]:
df = pd.concat([df, df_entreprise, df_JO], ignore_index=True)
df

,Question,Réponse,URL,Thème,Score de fiabilité
0,comment comprendre intuitivement mathématiques ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.877857
1,pourquoi ’ arrive comprendre math ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.872000
2,comment comprendre intuitivement mathématiques ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.862444
3,pourquoi ’ arrive comprendre math ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.898083
4,comment comprendre intuitivement mathématiques ?,entend souvent brillants mathématiciens expliq...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.804250
...,...,...,...,...,...
6672,Quels sont les sites de compétitions temporair...,Certains sites de compétitions seront temporai...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.979585
6673,Quels sont les sites de compétitions ?,"Il existe plus de 30 sites de compétition, rép...",https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,1.288634
6674,Comment rester alerté des éventuels changement...,Inscrivez-vous à l’alerte info pour rester inf...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.969000
6675,Où et quand auront lieu les épreuves des Jeux ?,Les Jeux Olympiques et Paralympiques de Paris ...,https://anticiperlesjeux.gouv.fr/faq?page=6,Jeux Olympiques,0.893000


In [80]:
df['Réponse'].nunique()

1343

In [88]:
#Moyenne des scores de fiabilité
df['Score de fiabilité'].mean()


0.8777789632988184

In [81]:
df.to_csv('faq.csv', index=False)

In [82]:
#formatage du jeu de données en vue d'une utilisation dans un chatbot
df = pd.read_csv('faq.csv')
df['Question'] = df['Question'].str.lower()
df['Réponse'] = df['Réponse'].str.lower()
df['Thème'] = df['Thème'].str.lower()
df['URL'] = df['URL'].str.lower()
df['Score de fiabilité'] = df['Score de fiabilité'].astype(float)
#suppression de la ponctuation
df['Question'] = df['Question'].str.replace(r'[^\w\s]','')
df['Réponse'] = df['Réponse'].str.replace(r'[^\w\s]','')
df


,Question,Réponse,URL,Thème,Score de fiabilité
0,comment comprendre intuitivement mathématiques ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.877857
1,pourquoi ’ arrive comprendre math ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.872000
2,comment comprendre intuitivement mathématiques ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.862444
3,pourquoi ’ arrive comprendre math ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.898083
4,comment comprendre intuitivement mathématiques ?,entend souvent brillants mathématiciens expliq...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.804250
...,...,...,...,...,...
6672,quels sont les sites de compétitions temporair...,certains sites de compétitions seront temporai...,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.979585
6673,quels sont les sites de compétitions ?,"il existe plus de 30 sites de compétition, rép...",https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,1.288634
6674,comment rester alerté des éventuels changement...,inscrivez-vous à l’alerte info pour rester inf...,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.969000
6675,où et quand auront lieu les épreuves des jeux ?,les jeux olympiques et paralympiques de paris ...,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.893000


In [83]:
#tokenisation du jeu de données
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords

def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text(text):
    # Tokenisation
    words = word_tokenize(text)
    
    # Stop words
    stop_words = set(stopwords.words('french'))
    words = [word for word in words if word.lower() not in stop_words]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    
    return ' '.join(words)

df['Question'] = df['Question'].astype(str).apply(preprocess_text)
df['Réponse'] = df['Réponse'].astype(str).apply(preprocess_text)
df

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/guillaumerabeau/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/guillaumerabeau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/guillaumerabeau/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/guillaumerabeau/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Question,Réponse,URL,Thème,Score de fiabilité
0,comment comprendre intuitivement mathématiques ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.877857
1,pourquoi ’ arrive comprendre math ?,’ faut faire ’ identifié règles ’ appliquent o...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.872000
2,comment comprendre intuitivement mathématiques ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.862444
3,pourquoi ’ arrive comprendre math ?,chacun trouver voie . parmi presque toutes rép...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.898083
4,comment comprendre intuitivement mathématiques ?,entend souvent brillants mathématiciens expliq...,https://fr.quora.com/comment-comprendre-les-maths,mathématiques,0.804250
...,...,...,...,...,...
6672,quels site compétitions temporaires ?,certains site compétitions temporaires paris ....,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.979585
6673,quels site compétitions ?,"existe plus 30 site compétition , répartis par...",https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,1.288634
6674,comment rester alerté éventuels changements co...,inscrivez-vous ’ alerte info rester informé de...,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.969000
6675,où quand lieu épreuves jeux ?,jeux olympiques paralympiques paris 2024 dérou...,https://anticiperlesjeux.gouv.fr/faq?page=6,jeux olympiques,0.893000


In [84]:
#Création des datasets de formation, de validation et de test 
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=42)

In [85]:
import torch
#Sauvegarde des données dans un format compatible (PyTorch)
torch.save(df_train, 'formatted_data.pt')
torch.save(df_val, 'formatted_data_val.pt')
torch.save(df_test, 'formatted_data_test.pt')
